# KMEANS

### 18/05/20

## Importing Libraries

In [5]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
#import scipy.signal as signal
from scipy.signal import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

import sys
#sys.path.insert(0, '/Users/SYL21/External_Drive/SUPAERO/PIR/AdabandFlt')
sys.path.insert(0, '/Users/louiseplacidet/Desktop/PIR/GITPIR/GIT_29_04/PIR/AdabandFlt')
#from AdaBandFlt import *
#from V2AdaBandFlt import *
from V3AdaBandFlt import *

%matplotlib tk

## Loading Data

In [6]:
#load data
# file path of csv file
#Location = r'/Users/33614/ExternalDrive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/D_Drive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/External_Drive/SUPAERO/PIR/Data/Wetransfer_data/E18KABaseline_BcutV2groundAll.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'

Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_new_spike_data/539W6cbaseRaw.txt'


# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')
df

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,El 41,El 42,El 44,El 54,El 55,El 56,El 58,El 86,El 15,El 51
%t,,,,,,,,,,
0.00,0.92,-1.46,2.32,-0.61,0.06,-1.04,8.06,-0.24,0.73,-0.55
0.02,0.49,-1.53,1.59,-0.31,0.73,-1.22,6.29,-0.85,0.92,-1.40
0.04,0.43,-1.34,0.92,-0.37,1.16,-1.10,4.15,-0.92,0.85,-2.08
0.06,0.85,-0.85,0.37,-0.85,1.04,-0.92,2.08,-0.73,0.61,-2.44
0.08,1.46,-0.37,-0.12,-1.46,0.31,-0.73,0.55,-0.55,0.18,-2.38
...,...,...,...,...,...,...,...,...,...,...
299999.90,1.34,0.73,4.21,-1.95,-4.76,-1.28,-6.96,-3.60,-0.31,2.50
299999.92,1.59,0.49,4.09,-2.44,-3.42,-1.34,-6.04,-4.09,0.00,2.93
299999.94,2.08,0.37,3.66,-3.48,-1.95,-1.53,-5.13,-4.64,0.18,2.93


 # Filtering the signal for all electrodes

### Global Parameters

In [7]:
## Parameters to choose from: 

size_of_data = 1000000
fs = 50000

method_align = 'indice_1er_depass'
time_before = 0.0015
time_after = 0.0015

n_electrode = 4

### Filtering Functions

In [8]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

### Filtering the electrode signals...

In [9]:
lowcut = 100.0
highcut = 5000.0

filtered_signal = []

for electrode in range(n_electrode):
    before_filter = df.iloc[:size_of_data,electrode]
    
    y = butter_bandpass_filter(before_filter, lowcut, highcut, fs, order=6)
    
    filtered_signal.append(y)
    
filtereddf_not_transposed = pd.DataFrame(filtered_signal)

    
filtereddf = filtereddf_not_transposed.transpose()
filtereddf.index = df.index[:size_of_data]
filtereddf.columns = df.columns[:n_electrode]

filtereddf

,El 41,El 42,El 44,El 54
%t,,,,
0.00,-0.655738,0.844880,0.203710,0.969816
0.02,-0.527641,0.924851,-0.546749,0.604473
0.04,-0.395290,1.032649,-1.233368,0.232775
0.06,-0.268779,1.204451,-1.803244,-0.115492
0.08,-0.179256,1.486950,-2.223309,-0.358307
...,...,...,...,...
19999.90,0.302640,-0.729945,0.729187,0.272311
19999.92,-0.028468,-0.692800,0.402893,0.885210
19999.94,-0.186762,-0.519373,0.196920,1.023643


# Detecting and Aligning the Spikes

### Setting Up Thresholds adapted to Noise Levels for each electrode

In [10]:
noise_levels_all_electrodes = []

for electrode in range(n_electrode):
    noise_levels = init_noise_levels(filtereddf.iloc[:size_of_data, electrode], fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 20,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5,
                      estimator_type = "RMS",
                      percentile_value = 25)
    noise_levels_all_electrodes.append(noise_levels)


In [11]:
# Plotting the noise levels for all electrodes
plt.figure()
for electrode in range(n_electrode):
    plt.plot(noise_levels_all_electrodes[electrode])
plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Noise Amplitude [µV]')
plt.title('Noise Levels')

Text(0.5, 1.0, 'Noise Levels')

### Detecting Spikes For Each Electrode

In [13]:
list_spike_info = []

for electrode in range(n_electrode):
    spike_info = find_spikes(filtereddf.iloc[:size_of_data, electrode], noise_levels_all_electrodes[electrode], fs, 
                          window_size = 0.001, 
                          noise_window_size = 0.01,
                          threshold_factor = 3.0,
                          maxseparation = 0.002)
    spike_info['Electrode num'] = electrode
    #Normalizing the amplitudes in a new column
    amplitude_norm = np.linalg.norm(spike_info['Delta_amplitudes'])
    spike_info['Delta_amp_norm'] = spike_info['Delta_amplitudes']/amplitude_norm
    list_spike_info.append(spike_info)

In [14]:
list_spike_info

[     indice_min  indice_1er_depass indice_max_gauche indice_max_droite  \
 0           144                144               nan               234   
 1          2567               2566              2556               nan   
 2         10959              10959             10934               nan   
 3         13728              13727               nan             13798   
 4         14075              14073             14066               nan   
 ..          ...                ...               ...               ...   
 337      990310             990308            990262            990375   
 338      992466             992465            992391               nan   
 339      997592             997591            997517               nan   
 340      998504             998503               nan            998535   
 341      999669             999668            999661               nan   
 
      Delta_amplitudes  Electrode num  Delta_amp_norm  
 0           12.073680              0     

### Recording the Spikes For Each Electrode

In [15]:
list_spike_data = []

for electrode in range(n_electrode):
    spike_data = record_spikes(filtereddf.iloc[:size_of_data, electrode], fs, list_spike_info[electrode],
                           method_align,
                           t_before = time_before, 
                           t_after = time_after)
    spike_data = spike_data.drop(columns=[0])
    list_spike_data.append(spike_data)

(343, 151)
(320, 151)
(326, 151)
(317, 151)


In [16]:
list_spike_data;

In [17]:
list_spike_data_normalized = []

for electrode in range(n_electrode):
    spike_data_normalized = record_spikes(filtereddf.iloc[:size_of_data, electrode], fs, list_spike_info[electrode],
                           method_align,
                           t_before = time_before, 
                           t_after = time_after)
    spike_data_normalized = spike_data_normalized.drop(columns=[0])
    for i in range(len(spike_data_normalized.columns)):
        amp_norm = np.linalg.norm(spike_data_normalized[i+1])
        spike_data_normalized[i+1] = spike_data_normalized[i+1]/amp_norm
    list_spike_data_normalized.append(spike_data_normalized)



(343, 151)
(320, 151)
(326, 151)
(317, 151)


In [18]:
list_spike_data_normalized;

In [19]:
list_spike_data_oneline = []

for electrode in range(n_electrode):
    spike_data_oneline = record_spikes_oneline(filtereddf.iloc[:size_of_data, electrode], fs, list_spike_info[electrode],
                                           method_align,
                                           t_before = time_before,
                                           t_after = time_after)
    list_spike_data_oneline.append(spike_data_oneline)

### Plotting the Spikes For Each Electrode

In [20]:
#Plotting the spikes on top of original signal

for electrode in range(n_electrode):
    plt.figure()
    plt.plot(filtereddf.index[:size_of_data], filtereddf.iloc[:size_of_data,electrode], color = 'blue')
    plt.plot(filtereddf.index[:size_of_data], list_spike_data_oneline[electrode], color = 'red')
    plt.title('Filtered Signal with Detected Spikes with RMS for electrode:'+str(filtereddf.columns[electrode]))
    plt.xlabel('Time Windows')
    plt.ylabel('Amplitude [µV]')
    plt.legend()
    plt.grid(True)

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [22]:
from random import randint

def print_spikes_adapted(spike_data,
                 t_before_alignement = 0,
                 first_spike = 1,
                 last_spike = -1,
                 fs = 25000,
                 randomize = False,
                 nb_spike = 20,
                 y_lim_min = -0.5,
                 y_lim_max = 0.5):
    
    if randomize == True:        
        kept = []
        m = len(spike_data.values[0])
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = spike_data.iloc[:,kept].values
        
    else:
        x = spike_data.iloc[:,first_spike:last_spike]
        
    figure = plt.figure()
    t_b = int(np.round(fs*(t_before_alignement)))
    axes = figure.add_subplot(1, 1, 1)
    axes.plot((spike_data.index-t_b)*1000/fs, x)
    axes.set_xlabel('Time in ms')
    axes.set_ylim(y_lim_min , y_lim_max)
    axes.grid()

In [23]:
# Plotting aligned spikes for each electrode
for electrode in range(n_electrode):
    print_spikes_adapted(list_spike_data_normalized[electrode],
             t_before_alignement = time_before,
             first_spike = 1,
             last_spike = 50,
             fs = fs)
    plt.title("Spikes aligned on "+str(method_align)+" for electrode "+str(filtereddf.columns[electrode]))

# Bilan PCA + KMEANS

### Merging spike_data and spike_info from all electrodes

In [24]:
spike_data_all_electrodes = pd.concat([list_spike_data[electrode] for electrode in range(n_electrode)], axis=1)
spike_data_all_electrodes.columns = [i+1 for i in range(len(spike_data_all_electrodes.columns))]
spike_data_all_electrodes

,1,2,3,4,5,6,7,8,9,10,...,1293,1294,1295,1296,1297,1298,1299,1300,1301,1302
0,-1.973703,4.220263,3.260929,-1.894941,-0.152721,0.848758,-0.516689,-1.031702,3.382428,2.018704,...,-1.986448,0.650114,1.885808,-2.209425,0.612260,2.397536,-1.701014,3.482424,0.500685,0.313315
1,-1.647949,4.121395,3.329283,-1.924816,-0.166782,0.240880,-0.607800,-1.626542,3.483976,2.791711,...,-2.170905,0.617540,1.236039,-2.510246,0.861803,3.737542,-1.926786,2.286266,-0.030227,-0.610352
2,-0.916940,3.694430,3.025160,-2.111020,-0.460642,-0.010334,-0.374542,-1.889271,2.982356,3.500228,...,-1.993842,0.550578,0.429259,-2.617527,0.903204,4.924290,-1.783059,0.819196,-0.411868,-1.285273
3,0.071935,2.994328,2.443338,-2.356010,-0.982586,0.094083,0.122580,-1.806691,1.957176,3.953124,...,-1.441861,0.554621,-0.377674,-2.430474,0.708066,5.804299,-1.292565,-0.653343,-0.580080,-1.568833
4,1.112113,2.123094,1.734555,-2.554203,-1.619872,0.511690,0.750325,-1.443280,0.621603,4.004366,...,-0.584274,0.676843,-1.024107,-1.911451,0.285485,6.267733,-0.531293,-1.884702,-0.513409,-1.426895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,1.431214,-1.275891,1.941138,6.195573,0.750988,0.618734,-3.181098,3.504759,-6.351350,2.141525,...,0.159259,5.438962,0.327499,-2.906062,-2.281847,3.165930,-0.204500,2.922067,4.714445,1.717421
147,1.952171,-0.940624,1.789257,6.096682,1.380654,0.226250,-2.381074,3.139431,-6.115506,1.659876,...,1.122536,5.047491,-0.278073,-2.622004,-2.741952,3.289785,0.570337,1.855687,3.802416,0.971946
148,2.387518,-0.430203,1.715605,5.807453,1.910447,-0.418538,-1.500399,2.431447,-5.719887,0.859322,...,2.097782,4.521962,-0.684017,-2.639121,-3.103294,3.062928,1.261382,0.887710,2.824129,0.484172
149,2.635003,0.193805,1.682434,5.304776,2.368949,-1.189761,-0.605397,1.539464,-5.201131,-0.108209,...,2.905942,3.978748,-0.958084,-2.954162,-3.214039,2.503904,1.825831,0.073250,1.905072,0.339117


In [25]:
list_spike_data_normalized

spike_data_norm_all_electrodes = pd.concat([list_spike_data_normalized[electrode] for electrode in range(n_electrode)], axis=1)
spike_data_norm_all_electrodes.columns = [i+1 for i in range(len(spike_data_norm_all_electrodes.columns))]
spike_data_norm_all_electrodes


,1,2,3,4,5,6,7,8,9,10,...,1293,1294,1295,1296,1297,1298,1299,1300,1301,1302
0,-0.057903,0.154644,0.093401,-0.050426,-0.003859,0.024503,-0.016429,-0.029422,0.092665,0.064342,...,-0.072448,0.017503,0.063440,-0.055985,0.018591,0.066949,-0.053728,0.113710,0.015807,0.008640
1,-0.048347,0.151021,0.095358,-0.051221,-0.004215,0.006954,-0.019326,-0.046386,0.095447,0.088980,...,-0.079175,0.016626,0.041581,-0.063608,0.026169,0.104368,-0.060859,0.074653,-0.000954,-0.016831
2,-0.026901,0.135376,0.086648,-0.056176,-0.011641,-0.000298,-0.011909,-0.053879,0.081705,0.111563,...,-0.072717,0.014823,0.014441,-0.066326,0.027426,0.137507,-0.056319,0.026749,-0.013003,-0.035442
3,0.002110,0.109722,0.069983,-0.062695,-0.024830,0.002716,0.003898,-0.051524,0.053619,0.125998,...,-0.052586,0.014932,-0.012705,-0.061586,0.021501,0.162080,-0.040827,-0.021333,-0.018314,-0.043262
4,0.032627,0.077797,0.049682,-0.067969,-0.040935,0.014772,0.023858,-0.041160,0.017029,0.127631,...,-0.021309,0.018222,-0.034452,-0.048435,0.008669,0.175021,-0.016781,-0.061540,-0.016209,-0.039348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,0.041988,-0.046753,0.055599,0.164868,0.018978,0.017863,-0.101148,0.099949,-0.174002,0.068257,...,0.005808,0.146431,0.011017,-0.073637,-0.069289,0.088406,-0.006459,0.095413,0.148843,0.047359
147,0.057272,-0.034468,0.051248,0.162236,0.034890,0.006532,-0.075710,0.089531,-0.167541,0.052905,...,0.040940,0.135891,-0.009355,-0.066439,-0.083260,0.091865,0.018015,0.060593,0.120049,0.026802
148,0.070044,-0.015764,0.049139,0.154540,0.048278,-0.012083,-0.047707,0.069340,-0.156702,0.027389,...,0.076508,0.121743,-0.023011,-0.066873,-0.094232,0.085530,0.039842,0.028986,0.089163,0.013351
149,0.077304,0.007102,0.048189,0.141163,0.059864,-0.034348,-0.019249,0.043903,-0.142490,-0.003449,...,0.105983,0.107118,-0.032231,-0.074856,-0.097595,0.069919,0.057670,0.002392,0.060146,0.009351


In [26]:
spike_info_all_electrodes = list_spike_info[0].append([list_spike_info[electrode+1] for electrode in range(n_electrode-1)])
spike_info_all_electrodes.index = [i+1 for i in range(len(spike_info_all_electrodes))]
spike_info_all_electrodes

,indice_min,indice_1er_depass,indice_max_gauche,indice_max_droite,Delta_amplitudes,Electrode num,Delta_amp_norm
1,144,144,nan,234,12.073680,0,0.050246
2,2567,2566,2556,nan,11.614876,0,0.048336
3,10959,10959,10934,nan,11.494119,0,0.047834
4,13728,13727,nan,13798,14.104975,0,0.058699
5,14075,14073,14066,nan,14.334066,0,0.059652
...,...,...,...,...,...,...,...
1298,987008,987006,986936,nan,13.674498,3,0.062151
1299,990511,990508,990425,990546,14.134667,3,0.064242
1300,991326,991326,nan,991393,11.978136,3,0.054441
1301,993078,993078,993069,nan,10.985894,3,0.049931


## PCA and KMEANS on spikes

In [27]:
def PCA_and_KMEANS_spikes_electrodes_combined(spike_data_norm_all_electrodes, spike_info_all_electrodes, nb_PCA_components=3, nb_clusters=3):
    
    ## PCA
    pca_data = np.array(spike_data_norm_all_electrodes.values).transpose()
    pca = PCA(n_components=nb_PCA_components)
    pca.fit(pca_data)
    PCA_X = pca.transform(pca_data)
    
    ## KMEANS
    kmeans = KMeans(n_clusters=nb_clusters, random_state=0).fit(PCA_X)

    
    ## Ajout du label des clusters dans spike info
    spike_info_all_electrodes['cluster_label'] = kmeans.labels_
    
    return PCA_X, kmeans, spike_info_all_electrodes
    

## Plotting the PCA

In [28]:
## Fonction qui plot la PCA

def PCA_plot(PCA_X):
    
    fig = plt.figure(4,figsize=(4,3))
    plt.clf()
    ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)
    plt.cla()
  
    ax.scatter(PCA_X[:, 0], PCA_X[:, 1], PCA_X[:, 2], cmap=plt.cm.nipy_spectral,
           edgecolor='k')

    ax.w_xaxis.set_ticklabels([])
    ax.w_yaxis.set_ticklabels([])
    ax.w_zaxis.set_ticklabels([])

    plt.show()

In [29]:
## Importance des premiers coefficients de la PCA selon les différents indices d'alignement

##ATTENTION! CETTE FONCTION N'EST APPLICABLE QU'À UNE SEULE ÉLECTRODE

def print_pca_explained_variance_ratio(signal, fs, spike_infos):
    legend = ['indice_max',
     'indice_min',
     'indice_depass_positif',
     'indice_depass_negatif',
     'indice_1er_depass',
     'indice_zero_central']
    explained_variance_ratio = []
    
    for method in legend:
        spike_data = record_spikes(signal, fs, spike_infos,
                  method,
                  t_before = 0.001,
                  t_after = 0.002)
        pca_data = np.array(spike_data.iloc[:,1:].values).transpose()
        pca = PCA(n_components=10)
        pca.fit(pca_data)
        X = pca.transform(pca_data)
        explained_variance_ratio.append(pca.explained_variance_ratio_)
        
    plt.figure()
    plt.plot(np.transpose(explained_variance_ratio))
    plt.legend(legend)

## Plotting the KMEANS

In [30]:
## Fonction qui plot le KMEANS

def print_clusters_3d(labels, PCA_X):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    nb_clusters = max(labels) + 1

    a = [i for i in range(len(labels))]
    b = np.transpose([a,list(labels)])

    for nb in range(nb_clusters):
        legend = 'Cluster n°'+str(nb)
        data = PCA_X[[x for x,y in b if y==nb],:]
        ax.scatter(data[:,0], data[:,1], data[:,2],label=legend)
    data = PCA_X[[x for x,y in b if y==-1],:]
    ax.scatter(data[:,0], data[:,1], data[:,2], c='black',label='Noise cluster')    
    
    ax.set_title('Nombre de cluster(s) :' + str(nb_clusters))
    ax.legend()
    plt.show()
    
#print_clusters_3d(kmeans.labels_, X)

## Plotting the spikes from the different clusters after KMEANS

In [31]:
## Fonction qui plot les spikes dans les clusters du KMEANS

def print_spikes_clusterized(spike_data_norm_all_electrodes,
                             labels,
                             t_before_alignement = 0,
                             nb_spike = 20,
                             y_lim_min = -50,
                             y_lim_max = 60,
                             fs = 25000):
    
    nb_clusters = max(labels) + 1
    
    if (-1 in labels) == True:
        nb_clusters_ = nb_clusters + 1
    else:
        nb_clusters_ = nb_clusters
    
    nb_line = nb_clusters_//2
    if nb_clusters_%2 != 0:
        nb_line += 1
    
    #spike_data.iloc[:,first_spike:last_spike].plot()
    t_b = int(np.round(fs*(t_before_alignement)))
    y = (spike_data_norm_all_electrodes.index-t_b)*1000/fs
        
    a = [i for i in range(len(labels))]
    b = np.transpose([a,list(labels)])
    
    figure = plt.figure()
    plt.gcf().subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.5)
    
    for nb in range(nb_clusters):
        
        data = spike_data_norm_all_electrodes.iloc[:,[x for x,z in b if z==nb]]
        m = len(data.values[0])
        nb_spikes_in_cluster = len(data.columns)
        
        
        kept = []
        
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = data.iloc[:,kept].values
        axes = figure.add_subplot(nb_line, 2, nb+1)
        axes.plot(y, x)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster numero ' + str(nb))
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid(True)
        axes.legend([str(nb_spikes_in_cluster)+' spikes'])
        
    if (-1 in labels) == True:
        data = spike_data_norm_all_electrodes.iloc[:,[x for x,y in b if y==-1]]
        m = len(data.values[0])
        nb_spikes_in_cluster = len(data.columns)

        
        kept = []
        
        if m <= nb_spike:
            kept = [i for i in range(m)]
        else:      
            i = 0  
            while i < nb_spike:
                r = randint(0,m-1)
                if (r in kept) == False:
                    kept.append(r)
                    i += 1
        
        x = data.iloc[:,kept].values
        
        axes = figure.add_subplot(nb_line, 2, nb+2)
        axes.plot(y, x)
        axes.set_xlabel('Time in ms')
        axes.set_title('Cluster de bruit')
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid(True)
        axes.legend([str(nb_spikes_in_cluster)+' spikes'])
        

#print_spikes_clusterized(spike_data,
#                             kmeans.labels_,
#                             t_before_alignement = 0.0015,
#                             nb_spike = 20,
#                             y_lim_min = -50,
#                             y_lim_max = 60,
#                             fs = 25000)

## Printing the spikes from clusters on original signal

In [32]:
## Fonction qui affiche les spikes des différents clusters sur échelle temporelle

def print_spikes_clusterized_oneline(filtereddf,
                                     updated_spike_info_all_electrodes,
                                     align_method = 'indice_zero_central',
                                     t_before = 0.001,
                                     t_after = 0.002,
                                     fs = 25000,
                                     y_lim_min = -50,
                                     y_lim_max = 60,
                                     separate_plot = False):
    
    electrodes = updated_spike_info_all_electrodes['Electrode num'].values
    nb_electrodes = max(electrodes) + 1
    
    for electrode in range(nb_electrodes):
        
        signal = filtereddf.iloc[:,electrode]
    
        labels = updated_spike_info_all_electrodes['cluster_label'].values
        nb_clusters = max(labels) + 1

        if (-1 in labels) == True:
            nb_clusters_ = nb_clusters + 2
        else:
            nb_clusters_ = nb_clusters + 1

        nb_line = nb_clusters_//2
        if nb_clusters_%2 != 0:
            nb_line += 1

        ghost_array = np.array(['NaN' for x in range(len(signal))])
        ghost_array = ghost_array.astype(float)
        spike_data_clusterized_oneline = []
        legend = ['Signald\'origine']

        figure = plt.figure()
        plt.title(str(filtereddf.columns[electrode]))
        plt.gcf().subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.5)
        for nb in range(nb_clusters):

            spike_data_onelinee = record_spikes_oneline(signal,
                                  fs,
                                  updated_spike_info_all_electrodes.loc[(updated_spike_info_all_electrodes['cluster_label'] == nb) & (updated_spike_info_all_electrodes['Electrode num'] == electrode)],
                                  align_method,
                                  t_before = t_before,
                                  t_after = t_after)
            spike_data_onelinee = np.resize(spike_data_onelinee.values,len(spike_data_onelinee.values))
            spike_data_clusterized_oneline.append(spike_data_onelinee)


            axes = figure.add_subplot(nb_line, 2, nb+1)
            axes.plot(signal.index, signal.values)
            for ghost in range(nb):
                axes.plot(signal.index, ghost_array)
            axes.plot(signal.index, list_spike_data_oneline[electrode])
            axes.set_xlabel('Time in ms')
            axes.set_title('Cluster n°' + str(nb))
            axes.set_ylim(y_lim_min , y_lim_max)
            axes.grid()

            legend.append('Cluster n°' + str(nb))

        if (-1 in labels) == True:
            spike_data_onelinee = record_spikes_oneline(signal,
                                  fs,
                                  updated_spike_info_all_electrodes.loc[(updated_spike_info_all_electrodes['cluster_label'] == -1) & (updated_spike_info_all_electrodes['Electrode num'] == electrode)],
                                  align_method,
                                  t_before = t_before,
                                  t_after = t_after)
            spike_data_onelinee = np.resize(spike_data_onelinee.values,len(spike_data_onelinee.values))
            spike_data_clusterized_oneline.append(spike_data_onelinee)


            axes = figure.add_subplot(nb_line, 2, nb+2)
            axes.plot(signal.index, signal.values)
            axes.plot(signal.index, spike_data_oneline)
            axes.set_xlabel('Time in ms')
            axes.set_title('Cluster de bruit')
            axes.set_ylim(y_lim_min , y_lim_max)
            axes.grid()

            legend.append('Cluster de bruit')

            axes = figure.add_subplot(nb_line, 2, nb+3)
        else:
            axes = figure.add_subplot(nb_line, 2, nb+2)

        axes.plot(signal.index, signal.values)
        axes.plot(signal.index, np.transpose(spike_data_clusterized_oneline))
        axes.legend(legend)
        axes.set_xlabel('Time in ms')
        axes.set_title('Tous les clusters')
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid()
        
        
        
        

#print_spikes_clusterized_oneline(signal,
#                                 updated_spike_infos,
#                                 align_method = 'indice_zero_central',
#                                 t_before = 0.001,
#                                 t_after = 0.002,
#                                 fs = 25000,)

In [33]:
## Fonction qui affiche les spikes des différents clusters sur échelle temporelle

def print_spikes_clusterized_oneline_total(filtereddf,
                                     updated_spike_info_all_electrodes,
                                     align_method = 'indice_zero_central',
                                     t_before = 0.001,
                                     t_after = 0.002,
                                     fs = 25000,
                                     y_lim_min = -50,
                                     y_lim_max = 60,
                                     separate_plot = False):

    electrodes = updated_spike_info_all_electrodes['Electrode num'].values
    nb_electrodes = max(electrodes) + 1
    
    for electrode in range(nb_electrodes):
        
        signal = filtereddf.iloc[:,electrode]
        labels = updated_spike_info_all_electrodes['cluster_label'].values
        nb_clusters = max(labels) + 1

        if (-1 in labels) == True:
            nb_clusters_ = nb_clusters + 2
        else:
            nb_clusters_ = nb_clusters + 1

        nb_line = nb_clusters_//2
        if nb_clusters_%2 != 0:
            nb_line += 1

        ghost_array = np.array(['NaN' for x in range(len(signal))])
        ghost_array = ghost_array.astype(float)
        spike_data_clusterized_oneline = []
        legend = ['Signald\'origine']

        figure = plt.figure()

        plt.gcf().subplots_adjust(left = 0.1, bottom = 0.1, right = 0.9, top = 0.9, wspace = 0.2, hspace = 0.5)
        
        for nb in range(nb_clusters):

            spike_data_onelinee = record_spikes_oneline(signal,
                                  fs,
                                  updated_spike_info_all_electrodes.loc[(updated_spike_info_all_electrodes['cluster_label'] == nb) & (updated_spike_info_all_electrodes['Electrode num'] == electrode)],
                                  align_method,
                                  t_before = t_before,
                                  t_after = t_after)
            spike_data_onelinee = np.resize(spike_data_onelinee.values,len(spike_data_onelinee.values))
            spike_data_clusterized_oneline.append(spike_data_onelinee)


            legend.append('Cluster n°' + str(nb))


        if (-1 in labels) == True:
            spike_data_onelinee = record_spikes_oneline(signal,
                                  fs,
                                  updated_spike_info_all_electrodes.loc[(updated_spike_info_all_electrodes['cluster_label'] == -1) & (updated_spike_info_all_electrodes['Electrode num'] == electrode)],
                                  align_method,
                                  t_before = t_before,
                                  t_after = t_after)
            spike_data_onelinee = np.resize(spike_data_onelinee.values,len(spike_data_onelinee.values))
            spike_data_clusterized_oneline.append(spike_data_onelinee)
            legend.append('Cluster de bruit')

        axes = figure.add_subplot(1, 1, 1)
        axes.plot(signal.index, signal.values)
        axes.plot(signal.index, np.transpose(spike_data_clusterized_oneline))
        axes.legend(legend)
        axes.set_xlabel('Time in ms')
        axes.set_title('Tous les clusters pour: '+str(filtereddf.columns[electrode]))
        axes.set_ylim(y_lim_min , y_lim_max)
        axes.grid()
        
        
        
        

#print_spikes_clusterized_oneline_total(signal,
#                                 updated_spike_infos,
#                                 align_method = 'indice_zero_central',
#                                 t_before = 0.001,
#                                 t_after = 0.002,
#                                 fs = 25000,)

## Histogram of Amplitudes and Time

In [34]:
## Histogramme de différences d'amplitude Peak-to-Peak des spikes

def histogram_spikes_amplitude(spike_info_all_electrodes):
    plt.figure()
    plt.hist(spike_info_all_electrodes['Delta_amplitudes'], bins='auto')
    plt.title("Histogram of Spike Peak-to-Peak Amplitude")
    plt.show()

In [35]:
## Histogramme de l'écartement temporel des spikes (imax-imin)

def histogram_spikes_time(spike_info_all_electrodes):
    plt.figure()
    plt.hist(spike_info_all_electrodes['i_max-i_min'], bins='auto')
    plt.title("Histogram of Spike i_max-i_min")
    plt.show()

In [36]:
## Histogramme des différences d'amplitude Peak-to-Peak des spikes:

## ¡¡¡ATTENTION!!! Il faut que le spike info en entrée, soit mis à jour avec les labels des clusters

def histogram_amplitude_clusterized(updated_spike_info_all_electrodes):
    
    labels = updated_spike_info_all_electrodes['cluster_label'].values
    nb_clusters = max(labels) + 1
    
    for nb in range(nb_clusters):
        local_info = updated_spike_info_all_electrodes.loc[updated_spike_info_all_electrodes['cluster_label'] == nb]
        plt.figure()
        plt.hist(local_info['Delta_amplitudes'], bins='auto')
        plt.title("Histogram of Spike peak-to-peak of cluster n°"+str(nb))
        plt.show()
    
    if(-1 in labels) == True:
        local_info = updated_spike_info_all_electrodes.loc[updated_spike_info_all_electrodes['cluster_label']==-1]
        plt.figure()
        plt.hist(local_info['Delta_amplitudes'], bins='auto')
        plt.title("Histogram of Spike peak-to-peak of cluster of noise")
        plt.show()

In [37]:
## Histogramme des différences d'amplitude Peak-to-Peak des spikes:

## ¡¡¡ATTENTION!!! Il faut que le spike info en entrée, soit mis à jour avec les labels des clusters

def histogram_amplitude_clusterized_superposed(updated_spike_info_all_electrodes):
    
    labels = updated_spike_info_all_electrodes['cluster_label'].values
    nb_clusters = max(labels) + 1
    
    for nb in range(nb_clusters,0,-1):
        local_info = updated_spike_info_all_electrodes.loc[updated_spike_info_all_electrodes['cluster_label'] == nb]
        plt.hist(local_info['Delta_amplitudes'], bins='auto')
        plt.title("Superposition of histogram of Spike peak-to-peak of clusters")
        plt.show()
    
    if(-1 in labels) == True:
        local_info = updated_spike_info_all_electrodes.loc[updated_spike_info_all_electrodes['cluster_label']==-1]
        plt.hist(local_info['Delta_amplitudes'], bins='auto')
        plt.show()

# Tests des fonctions

In [38]:
# Function: PCA_and_KMEANS_spikes_electrodes_combined(spike_data_all_electrodes, spike_info_all_electrodes, nb_PCA_componefnts=3, nb_clusters=3)

PCA_X, KMEANS, updated_spike_info_all_electrodes = PCA_and_KMEANS_spikes_electrodes_combined(spike_data_norm_all_electrodes, spike_info_all_electrodes, 5, 6)


In [39]:
# Function: PCA_plot(spike_data, nb_clusters=3)

PCA_plot(PCA_X)

In [40]:
# Function: print_clusters_3d(labels, PCA_X)

print_clusters_3d(KMEANS.labels_, PCA_X)

In [41]:
# Function: print_spikes_clusterized(spike_data_all_electrodes,
#                             labels,
#                             t_before_alignement = 0,
#                             nb_spike = 20,
#                             y_lim_min = -50,
#                             y_lim_max = 60,
#                             fs = 25000)
                        
print_spikes_clusterized(spike_data_norm_all_electrodes,
                             KMEANS.labels_,
                             t_before_alignement = time_before,
                             nb_spike = 20,
                             y_lim_min = -0.5,
                             y_lim_max = 0.5,
                             fs = fs)                        

/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)


In [42]:
# Function: print_spikes_clusterized_oneline(filtereddf,
#                                     updated_spike_info_all_electrodes,
#                                     align_method = 'indice_zero_central',
#                                     t_before = 0.001,
#                                     t_after = 0.002,
#                                     fs = 25000,
#                                     y_lim_min = -50,
#                                     y_lim_max = 60,
#                                     separate_plot = False)

print_spikes_clusterized_oneline(filtereddf,
                                 updated_spike_info_all_electrodes,
                                 align_method = method_align,
                                 t_before = time_before,
                                 t_after = time_after,
                                 fs = fs,)

In [43]:
# Function : print_spikes_clusterized_oneline_total(filtereddf,
#                                     updated_spike_info_all_electrodes,
#                                     align_method = 'indice_zero_central',
#                                     t_before = 0.001,
#                                     t_after = 0.002,
#                                     fs = 25000,
#                                     y_lim_min = -50,
#                                     y_lim_max = 60,
#                                     separate_plot = False)

print_spikes_clusterized_oneline_total(filtereddf,
                                 updated_spike_info_all_electrodes,
                                 align_method = method_align,
                                 t_before = time_before,
                                 t_after = time_after,
                                 fs = fs,)

In [44]:
# Function: histogram_spikes_amplitude(spike_info)

histogram_spikes_amplitude(updated_spike_info_all_electrodes)

In [45]:
# Function: histogram_spikes_time(spike_info)

histogram_spikes_time(updated_spike_info_all_electrodes)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


KeyError: 'i_max-i_min'

In [ ]:
# Function: histogram_amplitude_clusterized(spike_info)

histogram_amplitude_clusterized(updated_spike_info_all_electrodes)

/opt/anaconda3/lib/python3.7/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/opt/anaconda3/lib/python3.7/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [ ]:
# Function: histogram_amplitude_clusterized_superposed(updated_spike_info)

histogram_amplitude_clusterized_superposed(updated_spike_info_all_electrodes)

In [65]:
print_pca_explained_variance_ratio(signal, fs, spike_info)

(38, 151)
(38, 151)
(38, 151)
(38, 151)
(38, 151)
(38, 151)
